In [1]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import pandas as pd

In [2]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool, ChebConv
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [98]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 1000  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 2  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from sklearn.preprocessing import MinMaxScaler


x_data = []
y_data = []

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)
    
    x = np.array(x).astype("float32")
    for x_feat in x:
        x_data.append(x_feat)
        
    features = sp.csr_matrix(x).astype('float32')
    # adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    g = nx.DiGraph()
    g.add_nodes_from(graph.keys())
    for k, v in graph.items():
      g.add_edges_from(([(k, t) for t in v]))
      g.add_edges_from([(k, k)])
    adj = nx.adjacency_matrix(g)
    labels = np.array(y).astype('float32').reshape((-1,1))
    for l in y:
        y_data.append(l)
    print(adj.shape)
    print(features.shape)
    return adj, features, labels
  
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels


def _preprocess_features(features):
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.0
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features



class CircuitDataset(Dataset):
    def read(self):
        circuits = []
#         circs = ['c6288','c5315','c432', 'c499', 'c17', 'c880', 'c1355', 'c1908', 'c3540', 'adder.bench', 'arbiter.bench', 'cavlc.bench', 'dec.bench', 'voter.bench', "sin.bench","priority.bench", "multiplier.bench", "max.bench"]
        circs = ["adder.bench","arbiter.bench","c1355","c17","c1908","c3540","c432","c499","c5315","c6288","c880","cavlc.bench","dec.bench","int2float.bench","max.bench","multiplier.bench","priority.bench","sin.bench","voter.bench"]
#         circs = ["log2.bench"]
        for circ in circs:
            A, X, labels = load_data(circ, '../data/output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
            print(f"{circ}: {sum(labels)}, {len(labels)}")
        return circuits

def normalize_feature(circ_dataset):
    scaler = MinMaxScaler()
    scaler.fit(x_data)
    for graph in circ_dataset:
        graph.x = scaler.transform(graph.x)
    return circ_dataset

dataset = normalize_feature(CircuitDataset(transforms=[LayerPreprocess(ChebConv)]))

# Parameters

F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
np.random.seed(42)
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]
# dataset_tr = dataset[[8,6,5,1,2, 0, 3, 4, 7]]
# dataset_va = dataset[[0,3]]
# dataset_te = dataset[4,7]
# loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
# loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
# loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
def load_tr_data(epochs=400):
    loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
    loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
    loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
    return loader_tr, loader_va, loader_te

loader_tr, loader_va, loader_te = load_tr_data()


(4209, 4209)
(4209, 4)
adder.bench: [637.], 4209
(36352, 36352)
(36352, 4)
arbiter.bench: [3763.], 36352
(1355, 1355)
(1355, 4)
c1355: [192.], 1355
(17, 17)
(17, 4)
c17: [2.], 17
(1908, 1908)
(1908, 4)
c1908: [257.], 1908
(3540, 3540)
(3540, 4)
c3540: [406.], 3540
(432, 432)
(432, 4)
c432: [60.], 432
(499, 499)
(499, 4)
c499: [50.], 499
(5315, 5315)
(5315, 4)
c5315: [599.], 5315
(6288, 6288)
(6288, 4)
c6288: [846.], 6288
(880, 880)
(880, 4)
c880: [114.], 880
(2164, 2164)
(2164, 4)
cavlc.bench: [318.], 2164
(936, 936)
(936, 4)
dec.bench: [186.], 936
(761, 761)
(761, 4)
int2float.bench: [80.], 761
(9996, 9996)
(9996, 4)
max.bench: [1040.], 9996
(89943, 89943)
(89943, 4)
multiplier.bench: [9294.], 89943
(3495, 3495)
(3495, 4)
priority.bench: [350.], 3495
(16771, 16771)
(16771, 4)
sin.bench: [1723.], 16771
(50122, 50122)
(50122, 4)
voter.bench: [5012.], 50122
[ 0  5 11  1  8 16  3 13 15 17  2  9 18  4 12] [ 7 10] [14  6]


In [97]:
x_data

[array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1.,  1.,  1., inf], dtype=float32),
 array([ 1

In [99]:
N = sum([ g.n_nodes for g in dataset.graphs ])
n_out = 1
N

234983

In [100]:
from sklearn.utils.class_weight import compute_class_weight

y_data = []
for data_tr in dataset_tr:
    y_data.append(data_tr.y)


def _compute_class_weight_dictionary(y):
    # helper for returning a dictionary instead of an array
    classes = np.unique(y).astype('int')
    class_weight = compute_class_weight("balanced", classes, y)
    class_weight_dict = dict(zip(classes, class_weight))
    return class_weight_dict 

weights = _compute_class_weight_dictionary(np.vstack((y_data)).reshape((-1,)))
print(weights)

{0: 0.5593075068542587, 1: 4.715317980139447}


/Users/nikitaacharya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0 1], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [6]:
%load_ext tensorboard

In [76]:
from focal_loss import BinaryFocalLoss
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy

focal_loss_fn = BinaryFocalLoss(gamma=0.9999, pos_weight=1)

In [102]:
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy
channels = 256         # Number of channels in the first layer
dropout = 0.05         # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-3    # Learning rate
epochs = 500            # Number of training epochs
es_patience = 10    
patience = 100 # Patience for early stopping
# N = 

# Model definition
X_in = Input(shape=(F, ))
print("X_in: ", X_in.shape)
fltr_in = Input((None, ), sparse=True)
mask_in = Input((None, ))

# x_1 = BatchNormalization()(X_in)
# dropout_1 = Dropout(dropout)(x_1)
graph_conv_1 = ChebConv(512, K=2,
                         dropout_rate=dropout,
                         activation="relu",
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
#                          use_bias=False,
                         n_input_channels=dataset.n_node_features)([X_in, fltr_in])
x_2 = BatchNormalization()(graph_conv_1)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_2 = ChebConv(128, K=2,
                         dropout_rate=dropout,
                         activation="relu",
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
#                          use_bias=False,
                         n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])
x_2 = BatchNormalization()(graph_conv_2)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_3 = ChebConv(64, K=2,
                         dropout_rate=dropout,
                         activation="relu",
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
#                          use_bias=False,
                         n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])
x_2 = BatchNormalization()(graph_conv_3)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_4 = ChebConv(n_out,
                         activation='sigmoid',
                         n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])

model = Model(inputs=[X_in, fltr_in, mask_in], outputs=graph_conv_4)
optimizer = Adam(lr=learning_rate)

def masked_loss_and_accuracy(target, predictions):
    loss = binary_crossentropy(target, predictions)
    minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
    zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])
    ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])
    loss_zeroes =  tf.reduce_mean(tf.gather(loss, zeroes_index))
    loss_ones = tf.reduce_mean(tf.gather(loss, ones_index))
    
    weight_for_0 = tf.cast((1 / tf.shape(zeroes_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')
    weight_for_1 = tf.cast((1 / tf.shape(ones_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')

    return (weight_for_0 * loss_zeroes * 0.6 +  weight_for_1 * loss_ones * 0.4)


model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              weighted_metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
        log_dir='logs/WorkingModel',
        histogram_freq=1
)
# EarlyStopping(patience=patience, restore_best_weights=True)
callback_GCN = [tbCallBack_GCN]

X_in:  (None, 4)
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
cheb_conv_56 (ChebConv)         (None, 512)          4608        input_43[0][0]                   
                                                                 input_44[0][0]                   
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 512)          2048        cheb_conv

In [91]:
rm -rf ./DetectFaultInCircuit

In [ ]:
loader_tr, loader_va, loader_te = load_tr_data(epochs)
# class_weight = {0: 0.11, 1: 0.89}
history = model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    class_weight=weights,
    callbacks=callback_GCN
)

Epoch 1/500
8/8 [==============================] - 11s 875ms/step - loss: 0.7708 - acc: 0.6764 - precision_14: 0.6261 - recall_14: 0.8601 - val_loss: 0.8398 - val_acc: 0.4097 - val_precision_14: 0.1505 - val_recall_14: 0.8537
Epoch 2/500
8/8 [==============================] - 6s 584ms/step - loss: 1.4332 - acc: 0.4556 - precision_14: 0.4149 - recall_14: 0.3584 - val_loss: 0.8521 - val_acc: 0.1262 - val_precision_14: 0.1198 - val_recall_14: 1.0000
Epoch 3/500
8/8 [==============================] - 5s 706ms/step - loss: 0.8163 - acc: 0.6440 - precision_14: 0.6494 - recall_14: 0.6950 - val_loss: 0.8772 - val_acc: 0.1247 - val_precision_14: 0.1196 - val_recall_14: 1.0000
Epoch 4/500
8/8 [==============================] - 7s 264ms/step - loss: 0.7696 - acc: 0.6370 - precision_14: 0.5984 - recall_14: 0.8158 - val_loss: 0.8766 - val_acc: 0.1327 - val_precision_14: 0.1206 - val_recall_14: 1.0000
Epoch 5/500
8/8 [==============================] - 5s 596ms/step - loss: 0.7272 - acc: 0.7224 - pre

8/8 [==============================] - 5s 727ms/step - loss: 0.7055 - acc: 0.7351 - precision_14: 0.7984 - recall_14: 0.6875 - val_loss: 0.5406 - val_acc: 0.8390 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 37/500
8/8 [==============================] - 5s 650ms/step - loss: 0.5183 - acc: 0.8478 - precision_14: 0.7988 - recall_14: 0.9299 - val_loss: 0.5261 - val_acc: 0.8390 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 38/500
8/8 [==============================] - 6s 792ms/step - loss: 0.5513 - acc: 0.8224 - precision_14: 0.8048 - recall_14: 0.8859 - val_loss: 0.5305 - val_acc: 0.8390 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 39/500
8/8 [==============================] - 5s 554ms/step - loss: 0.6312 - acc: 0.7046 - precision_14: 0.7045 - recall_14: 0.6313 - val_loss: 0.5141 - val_acc: 0.8390 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 40/500
8/8 [==============================] - 5s 603ms/step - loss: 0.4

8/8 [==============================] - 6s 836ms/step - loss: 0.4701 - acc: 0.8619 - precision_14: 0.8656 - recall_14: 0.8820 - val_loss: 0.4750 - val_acc: 0.8840 - val_precision_14: 0.5417 - val_recall_14: 0.1585
Epoch 72/500
8/8 [==============================] - 5s 583ms/step - loss: 0.4878 - acc: 0.8495 - precision_14: 0.8064 - recall_14: 0.9212 - val_loss: 0.4862 - val_acc: 0.8840 - val_precision_14: 0.5417 - val_recall_14: 0.1585
Epoch 73/500
8/8 [==============================] - 5s 650ms/step - loss: 0.4660 - acc: 0.8526 - precision_14: 0.8385 - recall_14: 0.8688 - val_loss: 0.4825 - val_acc: 0.8840 - val_precision_14: 0.5417 - val_recall_14: 0.1585
Epoch 74/500
8/8 [==============================] - 5s 721ms/step - loss: 0.5144 - acc: 0.8302 - precision_14: 0.8088 - recall_14: 0.8737 - val_loss: 0.4754 - val_acc: 0.8840 - val_precision_14: 0.5417 - val_recall_14: 0.1585
Epoch 75/500
8/8 [==============================] - 5s 689ms/step - loss: 0.4618 - acc: 0.8753 - precision_14

In [82]:
%tensorboard --logdir logs/WorkingModel

In [83]:
model_loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(model_loss))

2/2 [==============================] - 0s 19ms/step - loss: 0.8230 - acc: 0.8841 - precision_12: 0.2358 - recall_12: 0.0433
Done. Test loss: [0.8230414986610413, 0.8840891718864441, 0.23579545319080353, 0.04334203526377678]


In [85]:
from sklearn.metrics import classification_report
print(len(dataset_te))
test_data = dataset_te[1]
X_te = test_data.x
A_te = test_data.a
y_te = test_data.y

y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, (y_pred >= 0.5).astype(int))
print('GCN Classification Report: \n {}'.format(report))

2
GCN Classification Report: 
               precision    recall  f1-score   support

         0.0       0.88      0.77      0.82      1163
         1.0       0.20      0.33      0.25       192

    accuracy                           0.71      1355
   macro avg       0.54      0.55      0.53      1355
weighted avg       0.78      0.71      0.74      1355



In [87]:
from sklearn.metrics import classification_report
print(len(dataset_va))
test_data = dataset_va[0]
X_te = test_data.x
A_te = test_data.a
y_te = test_data.y

y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, (y_pred >= 0.5).astype(int))
print('GCN Classification Report: \n {}'.format(report))

2
GCN Classification Report: 
               precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      1651
         1.0       0.75      0.59      0.66       257

    accuracy                           0.92      1908
   macro avg       0.84      0.78      0.81      1908
weighted avg       0.91      0.92      0.91      1908

